In [1]:
import tensorflow as tf
import numpy as np

In [2]:
epochs = 10
batch = 256

In [3]:
def build_cnn():
    inputs = tf.keras.Input(shape=(28,28,1))
    x = tf.keras.layers.Conv2D(8,(3,3),padding='same',activation='relu')(inputs)
    x = tf.keras.layers.Flatten()(x)
    outputs = tf.keras.layers.Dense(10,activation='softmax')(x)
    model = tf.keras.Model(inputs,outputs)
    return model

## Using numpy array as inputs

In [4]:
(X_train_1,y_train_1),(X_test_1,y_test_1) = tf.keras.datasets.mnist.load_data()
X_train_1 = X_train_1.astype(np.float32)/255.
X_train_1 = np.expand_dims(X_train_1,axis=-1)
X_test_1 = X_test_1.astype(np.float32)/255.
X_test_1 = np.expand_dims(X_test_1,axis=-1)
y_train_1 = tf.keras.utils.to_categorical(y_train_1,10)
y_test_1 = tf.keras.utils.to_categorical(y_test_1,10)

model_1 = build_cnn()
model_1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_1.fit(X_train_1,y_train_1,epochs=epochs,batch_size=batch,validation_data=(X_test_1,y_test_1),verbose=2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 - 2s - loss: 0.4546 - acc: 0.8805 - val_loss: 0.2198 - val_acc: 0.9382
Epoch 2/10
60000/60000 - 2s - loss: 0.1915 - acc: 0.9468 - val_loss: 0.1539 - val_acc: 0.9569
Epoch 3/10
60000/60000 - 2s - loss: 0.1375 - acc: 0.9621 - val_loss: 0.1174 - val_acc: 0.9663
Epoch 4/10
60000/60000 - 2s - loss: 0.1105 - acc: 0.9700 - val_loss: 0.1022 - val_acc: 0.9704
Epoch 5/10
60000/60000 - 2s - loss: 0.0929 - acc: 0.9743 - val_loss: 0.0925 - val_acc: 0.9726
Epoch 6/10
60000/60000 - 2s - loss: 0.0807 - acc: 0.9782 - val_loss: 0.0823 - val_acc: 0.9746
Epoch 7/10
60000/60000 - 2s - loss: 0.0699 - acc: 0.9807 - val_loss: 0.0803 - val_acc: 0.9753
Epoch 8/10
60000/60000 - 2s - loss: 0.0636 - acc: 0.9822 - val_loss: 0.0726 - val_acc: 0.9781
Epoch 9/10
60000/60000 - 2s - loss: 0.0567 - acc: 0.9844 - val_loss: 0.0738 - va

In [5]:
model_1.evaluate(X_test_1,y_test_1)

10000/10000 [==============================] - 0s 26us/sample - loss: 0.0665 - acc: 0.9787


[0.06648708410989493, 0.9787]

## Using tf.data.Dataset as inputs

In [6]:
def preprocessing(img,label):
    img = tf.cast(img,tf.float64)
    img = img/255.
    img = tf.expand_dims(img,axis=-1)
    label = tf.one_hot(label,10,dtype=tf.int32)
    return img,label

In [7]:
(X_train_2,y_train_2),(X_test_2,y_test_2) = tf.keras.datasets.mnist.load_data()

train_ds = tf.data.Dataset.from_tensor_slices((X_train_2,y_train_2)).map(preprocessing).shuffle(256).batch(batch).repeat(epochs)
test_ds = tf.data.Dataset.from_tensor_slices((X_test_2,y_test_2)).map(preprocessing).batch(batch).repeat(epochs)

model_2 = build_cnn()
model_2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_2.fit(train_ds,epochs=epochs,steps_per_epoch=len(X_train_2)//batch,validation_data=test_ds,validation_steps=5,verbose=2)

Epoch 1/10
234/234 - 4s - loss: 0.4850 - acc: 0.8755 - val_loss: 0.3094 - val_acc: 0.9031
Epoch 2/10
234/234 - 4s - loss: 0.2355 - acc: 0.9343 - val_loss: 0.2263 - val_acc: 0.9336
Epoch 3/10
234/234 - 4s - loss: 0.1748 - acc: 0.9520 - val_loss: 0.1785 - val_acc: 0.9438
Epoch 4/10
234/234 - 4s - loss: 0.1359 - acc: 0.9633 - val_loss: 0.1505 - val_acc: 0.9539
Epoch 5/10
234/234 - 4s - loss: 0.1105 - acc: 0.9705 - val_loss: 0.1334 - val_acc: 0.9594
Epoch 6/10
234/234 - 4s - loss: 0.0942 - acc: 0.9744 - val_loss: 0.1206 - val_acc: 0.9625
Epoch 7/10
234/234 - 4s - loss: 0.0817 - acc: 0.9779 - val_loss: 0.1107 - val_acc: 0.9656
Epoch 8/10
234/234 - 4s - loss: 0.0728 - acc: 0.9798 - val_loss: 0.1037 - val_acc: 0.9695
Epoch 9/10
234/234 - 4s - loss: 0.0650 - acc: 0.9818 - val_loss: 0.0985 - val_acc: 0.9719
Epoch 10/10
234/234 - 4s - loss: 0.0595 - acc: 0.9833 - val_loss: 0.0960 - val_acc: 0.9727


In [8]:
model_2.evaluate(test_ds)

400/400 [==============================] - 5s 12ms/step - loss: 0.0731 - acc: 0.9773


[0.07309556936379522, 0.9773]

## tensorflow distributed training with Keras

In [9]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4621682105683667972
]


In [10]:
(X_train_3,y_train_3),(X_test_3,y_test_3) = tf.keras.datasets.mnist.load_data()

mirrored = tf.distribute.MirroredStrategy()
print('\nnumber of replicas in sync: {}'.format(mirrored.num_replicas_in_sync))

batch_per_replica = batch
global_batch = batch_per_replica * mirrored.num_replicas_in_sync
print('global batch: {}'.format(global_batch))


number of replicas in sync: 1
global batch: 256


In [11]:
train_ds_dist = tf.data.Dataset.from_tensor_slices((X_train_3,y_train_3)).map(preprocessing).shuffle(256).batch(global_batch).repeat(epochs)
test_ds_dist = tf.data.Dataset.from_tensor_slices((X_test_3,y_test_3)).map(preprocessing).batch(global_batch).repeat(epochs)

with mirrored.scope():
    model_3 = build_cnn()
    model_3.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_3.fit(train_ds_dist,epochs=epochs,steps_per_epoch=len(X_train_3)//batch,validation_data=test_ds_dist,validation_steps=5,verbose=2)

INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CP

In [12]:
model_3.evaluate(test_ds_dist)

400/400 [==============================] - 5s 13ms/step - loss: 0.0772 - acc: 0.9756


[0.07716921950923279, 0.9756]